In [1]:
import pandas as pd
from sqlalchemy import create_engine


### Store CSV into DataFrame

In [2]:
from secrets import username, password


In [28]:
csv_file = "../kroger/kroger_milk_prices.csv"
kroger_data_df = pd.read_csv(csv_file)
kroger_data_df.head()

,Unnamed: 0,productId,categories,brand,description,size,price.regular,price.promo,Date,Store_Id,image
0,0,1111040101,['Dairy'],Kroger,Kroger® Vitamin D Whole Milk,1 gal,2.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
1,1,1111041600,['Dairy'],Kroger,Kroger® 2% Reduced Fat Milk,1/2 gal,1.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
2,2,1111041660,['Dairy'],Kroger,Kroger® 1% Lowfat Milk,1 gal,2.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
3,3,1111040601,['Dairy'],Kroger,Kroger® Vitamin D Whole Milk,1/2 gal,1.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...
4,4,1111042315,['Dairy'],Kroger,Kroger® Fat Free Skim Milk,1 gal,2.99,0.0,02/08/2021,3400312,https://www.kroger.com/product/images/xlarge/f...


In [39]:
milk_df = kroger_data_df
milk_df = milk_df.drop(columns=['Unnamed: 0', 'description' , 'price.regular', 'price.promo', 'Date', 'Store_Id'])
features = 'none'
milk_df['features'] = features
# milk_type = 1
# milk_df['Type_ID'] = milk_type
milk_df = milk_df.rename(columns={"productId": "product_id", "brand": "Brand" , "categories" : "category"})
milk_df.head()

,product_id,category,Brand,size,image,features
0,1111040101,['Dairy'],Kroger,1 gal,https://www.kroger.com/product/images/xlarge/f...,none
1,1111041600,['Dairy'],Kroger,1/2 gal,https://www.kroger.com/product/images/xlarge/f...,none
2,1111041660,['Dairy'],Kroger,1 gal,https://www.kroger.com/product/images/xlarge/f...,none
3,1111040601,['Dairy'],Kroger,1/2 gal,https://www.kroger.com/product/images/xlarge/f...,none
4,1111042315,['Dairy'],Kroger,1 gal,https://www.kroger.com/product/images/xlarge/f...,none


In [35]:
store_df = kroger_data_df[['Store_Id']].copy()
zipcode = 77007
store_df['Store_Zipcode'] = zipcode
name = "Kroger"
store_df['Store_Name'] = name
store_df

,Store_Id,Store_Zipcode,Store_Name
0,3400312,77007,Kroger
1,3400312,77007,Kroger
2,3400312,77007,Kroger
3,3400312,77007,Kroger
4,3400312,77007,Kroger
...,...,...,...
995,3400312,77007,Kroger
996,3400312,77007,Kroger
997,3400312,77007,Kroger
998,3400312,77007,Kroger


In [47]:
milk_type_df = pd.DataFrame()
milk_type = ['1%' , "2%", 'whole', 'skim', 'almond', 'other']
milk_type_df['Type'] = milk_type
milk_type_df.head()

,Type
0,1%
1,2%
2,whole
3,skim
4,almond


In [48]:
price_history_df = kroger_data_df[['Date', 'price.regular', 'price.promo']].copy()
price_history_df =price_history_df.rename(columns={
price_history_df.head()

,Date,price.regular,price.promo
0,02/08/2021,2.99,0.0
1,02/08/2021,1.99,0.0
2,02/08/2021,2.99,0.0
3,02/08/2021,1.99,0.0
4,02/08/2021,2.99,0.0


### Connect to local database

In [17]:
rds_connection_string = f"{username}:{password}@localhost:5432/milk_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [18]:
engine.table_names()

['Store', 'Milk', 'Milk_Type', 'Price_History']

### Use pandas to load csv converted DataFrame into database

In [13]:
new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

### Use pandas to load json converted DataFrame into database

In [14]:
new_customer_location_df.to_sql(name='customer_location', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [15]:
pd.read_sql_query('select * from customer_name', con=engine).head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Confirm data has been added by querying the customer_location table

In [16]:
pd.read_sql_query('select * from customer_location', con=engine).head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York
